In [1]:
# LSTM and RNN code derived from the following github repo: https://github.com/TobiasLee/Text-Classification

import numpy as np
import pandas as pd
import time
import pickle

from tensorflow.contrib.rnn import BasicLSTMCell
from tensorflow.python.ops.rnn import bidirectional_dynamic_rnn as bi_rnn
import tensorflow as tf
tf.disable_v2_behavior()

from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

c:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\Asus\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\Asus\AppData\

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# FOR FINAL TEMPORAL PRECISION EXPERIMENTS
input311File = "../../Comparison/TEMPORAL_PRECISION/DEEPCRIME/Dataset/1d/complaint_matrix_1d.pkl"
inputCrimeFile = "../../Comparison/TEMPORAL_PRECISION/DEEPCRIME/Dataset/1d/crime_matrix_1d.pkl"

with open(input311File, 'rb') as pickle_file:
    anomaly = pickle.load(pickle_file)

with open(inputCrimeFile,'rb') as pickle_file:
    content = pickle.load(pickle_file)

In [3]:
test_upto_MARCH = pickle.load(open("../Chicago Dataset/test_upto_MARCH.pkl", "rb"))
test_xmas = pickle.load(open("../Chicago Dataset/test_xmas.pkl", "rb"))
test_easter = pickle.load(open("../Chicago Dataset/test_upto_MARCH.pkl", "rb"))

In [4]:
inp = np.reshape(content, (365, 77*4))
inpA = np.reshape(anomaly, (365, 77*4))

inp1 = np.where(inp > 0, 1, 0)
print(np.shape(inp1))
print(np.shape(inpA))

(365, 308)
(365, 308)


In [45]:
# Train test split for the above data

# size = int(len(inp)*0.8)
size = 91
x_train = inp
y_train = inp1

# x_test = np.reshape(test_upto_MARCH, (len(test_upto_MARCH), 77*4))
x_test = np.reshape(test_easter, (len(test_easter), 77*4))
# x_test = np.reshape(test_xmas, (len(test_xmas), 77*4))
y_test = np.where(x_test > 0, 1, 0)

x_train2 = inpA
x_test2 = inpA[:size]

print(np.shape(x_train))
print(np.shape(y_train))
print(np.shape(x_test))
print(np.shape(y_test))
print(np.shape(x_train2))
print(np.shape(x_test2))

(365, 308)
(365, 308)
(91, 308)
(91, 308)
(365, 308)
(91, 308)


In [46]:
def attention(inputs, attention_size, time_major=False, return_alphas=False):
    """
    Attention mechanism layer which reduces RNN/Bi-RNN outputs with Attention vector.
    The idea was proposed in the article by Z. Yang et al., "Hierarchical Attention Networks
     for Document Classification", 2016: http://www.aclweb.org/anthology/N16-1174.
    Variables notation is also inherited from the article
    Args:
        inputs: The Attention inputs.
            Matches outputs of RNN/Bi-RNN layer (not final state):
                In case of RNN, this must be RNN outputs `Tensor`:
                    If time_major == False (default), this must be a tensor of shape:
                        `[batch_size, max_time, cell.output_size]`.
                    If time_major == True, this must be a tensor of shape:
                        `[max_time, batch_size, cell.output_size]`.
                In case of Bidirectional RNN, this must be a tuple (outputs_fw, outputs_bw) containing the forward and
                the backward RNN outputs `Tensor`.
                    If time_major == False (default),
                        outputs_fw is a `Tensor` shaped:
                        `[batch_size, max_time, cell_fw.output_size]`
                        and outputs_bw is a `Tensor` shaped:
                        `[batch_size, max_time, cell_bw.output_size]`.
                    If time_major == True,
                        outputs_fw is a `Tensor` shaped:
                        `[max_time, batch_size, cell_fw.output_size]`
                        and outputs_bw is a `Tensor` shaped:
                        `[max_time, batch_size, cell_bw.output_size]`.
        attention_size: Linear size of the Attention weights.
        time_major: The shape format of the `inputs` Tensors.
            If true, these `Tensors` must be shaped `[max_time, batch_size, depth]`.
            If false, these `Tensors` must be shaped `[batch_size, max_time, depth]`.
            Using `time_major = True` is a bit more efficient because it avoids
            transposes at the beginning and end of the RNN calculation.  However,
            most TensorFlow data is batch-major, so by default this function
            accepts input and emits output in batch-major form.
        return_alphas: Whether to return attention coefficients variable along with layer's output.
            Used for visualization purpose.
    Returns:
        The Attention output `Tensor`.
        In case of RNN, this will be a `Tensor` shaped:
            `[batch_size, cell.output_size]`.
        In case of Bidirectional RNN, this will be a `Tensor` shaped:
            `[batch_size, cell_fw.output_size + cell_bw.output_size]`.
    """

    if isinstance(inputs, tuple):
        # In case of Bi-RNN, concatenate the forward and the backward RNN outputs.
        inputs = tf.concat(inputs, 2)

    if time_major:
        # (T,B,D) => (B,T,D)
        inputs = tf.array_ops.transpose(inputs, [1, 0, 2])

    hidden_size = inputs.shape[2].value  # D value - hidden size of the RNN layer

    # Trainable parameters
    w_omega = tf.Variable(tf.random_normal([hidden_size, attention_size], stddev=0.1))
    b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
    u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))

    with tf.name_scope('v'):
        # Applying fully connected layer with non-linear activation to each of the B*T timestamps;
        #  the shape of `v` is (B,T,D)*(D,A)=(B,T,A), where A=attention_size
        v = tf.tanh(tf.tensordot(inputs, w_omega, axes=1) + b_omega)

    # For each of the timestamps its vector of size A from `v` is reduced with `u` vector
    vu = tf.tensordot(v, u_omega, axes=1, name='vu')  # (B,T) shape
    alphas = tf.nn.softmax(vu, name='alphas')  # (B,T) shape

    # Output of (Bi-)RNN is reduced with attention vector; the result has (B,D) shape
    output = tf.reduce_sum(inputs * tf.expand_dims(alphas, -1), 1)

    if not return_alphas:
        return output
    else:
        return output, alphas

In [47]:

def split_dataset(x_test, y_test, dev_ratio):
    """split test dataset to test and dev set with ratio """
    test_size = len(x_test)
    print(test_size)
    dev_size = (int)(test_size * dev_ratio)
    print(dev_size)
    x_dev = x_test[:dev_size]
    x_test = x_test[dev_size:]
    y_dev = y_test[:dev_size]
    y_test = y_test[dev_size:]
    return x_test, x_dev, y_test, y_dev, dev_size, test_size - dev_size


def fill_feed_dict(data_X, data_Y, batch_size):
    """Generator to yield batches"""
    # Shuffle data first.
    shuffled_X, shuffled_Y = shuffle(data_X, data_Y)
    # print("before shuffle: ", data_Y[:10])
    # print(data_X.shape[0])
    # perm = np.random.permutation(data_X.shape[0])
    # data_X = data_X[perm]
    # shuffled_Y = data_Y[perm]
    # print("after shuffle: ", shuffled_Y[:10])
    for idx in range(data_X.shape[0] // batch_size):
        x_batch = shuffled_X[batch_size * idx: batch_size * (idx + 1)]
        y_batch = shuffled_Y[batch_size * idx: batch_size * (idx + 1)]
        yield x_batch, y_batch

In [48]:
# Hyperparameters

# # default setting
# MAX_DOCUMENT_LENGTH = 128
# EMBEDDING_SIZE = 128
# HIDDEN_SIZE = 64
# ATTENTION_SIZE = 64
# lr = 5e-4
# learning_rate=0.001
# hidden_dim = 250
# BATCH_SIZE = 4
# KEEP_PROB = 1.0
# LAMBDA = 0.0001
# MAX_LABEL = 77*4
# # MAX_LABEL = 5*4
# epochs = 10
# latent_dim = 8
# # n_batches = 1
# timeSize = 10
max_len=10

# # test setting
HIDDEN_SIZE = 64 * 2
lr = 5e-4
BATCH_SIZE = 10 
KEEP_PROB = 0.8
MAX_LABEL = 77*4
# MAX_LABEL = 5*4
epochs = 50
timeSize = 20 

In [49]:
def multi_label_hot(prediction, threshold=0.5):
    prediction = tf.cast(prediction, tf.float32)
    threshold = float(threshold)
    return tf.cast(tf.greater(prediction, threshold), tf.int64)

def get_metrics(labels_tensor, one_hot_prediction, num_classes):
    metrics = {}
    with tf.variable_scope("metrics"):
        for scope in ["train", "val"]:
            with tf.variable_scope(scope):
                with tf.variable_scope("accuracy"):
                    accuracy, accuracy_update = tf.metrics.accuracy(
                        tf.cast(one_hot_prediction, tf.int32),
                        labels_tensor,
                    )
                metrics[scope] = {
                    "accuracy": accuracy,
                    "updates": tf.group(accuracy_update),
                }
    return metrics

In [50]:
# RNN based architecture with Attention
# https://github.com/TobiasLee/Text-Classification

tf.reset_default_graph()
batch_x = tf.placeholder(tf.float32, [None,timeSize,MAX_LABEL])
anomaly_x = tf.placeholder(tf.float32, [None,timeSize,MAX_LABEL])
batch_y = tf.placeholder(tf.float32, [None, MAX_LABEL])
keep_prob = tf.placeholder(tf.float32)

rnn_outputs1, _ = tf.nn.dynamic_rnn(BasicLSTMCell(HIDDEN_SIZE),
                        inputs=batch_x, dtype=tf.float32,scope='BLSTM_1')

rnn_outputs2, _ = tf.nn.dynamic_rnn(BasicLSTMCell(HIDDEN_SIZE),
                        inputs=anomaly_x, dtype=tf.float32,scope='BLSTM_2')

# weights for balance-outs
weight_out = tf.Variable(tf.truncated_normal([2], stddev=0.1))
weight_soft = tf.nn.softmax(weight_out)

inputAdd = weight_soft[0]*rnn_outputs1 + weight_soft[1]*rnn_outputs2
print(batch_x.shape)
print(inputAdd.shape)
rnn_outputs, _ = tf.nn.dynamic_rnn(BasicLSTMCell(HIDDEN_SIZE),
                        inputs=inputAdd, dtype=tf.float32,scope='BLSTM_3')
# fw_outputs, bw_outputs = rnn_outputs


# # Attention
# attention_output, alphas = attention(rnn_outputs, ATTENTION_SIZE, return_alphas=True)
# drop = tf.nn.dropout(attention_output, keep_prob)
# shape = drop.get_shape()
# print(shape)
# # Fully connected layer（dense layer)
# W = tf.Variable(tf.truncated_normal([shape[1].value, MAX_LABEL], stddev=0.1))
# b = tf.Variable(tf.constant(0., shape=[MAX_LABEL]))
# y_hat = tf.nn.xw_plus_b(drop, W, b)
W = tf.Variable(tf.random_normal([HIDDEN_SIZE], stddev=0.1))
H = rnn_outputs  # (batch_size, seq_len, HIDDEN_SIZE)
M = tf.tanh(H)  # M = tanh(H)  (batch_size, seq_len, HIDDEN_SIZE)

alpha = tf.nn.softmax(tf.reshape(tf.matmul(tf.reshape(M, [-1, HIDDEN_SIZE]),
                                                tf.reshape(W, [-1, 1])),
                                      (-1, timeSize )))  # batch_size x seq_len

print(alpha.shape)
r = tf.matmul(tf.transpose(H, [0, 2, 1]),
              tf.reshape(alpha, [-1, timeSize, 1]))
r = tf.squeeze(r)
h_star = tf.tanh(r)  # (batch , HIDDEN_SIZE

h_drop = tf.nn.dropout(h_star, keep_prob)
shape = h_drop.get_shape()
# print(h_star.shape)
# Fully connected layer（dense layer)
FC_W = tf.Variable(tf.truncated_normal([HIDDEN_SIZE, MAX_LABEL], stddev=0.1))
FC_b = tf.Variable(tf.constant(0., shape=[MAX_LABEL]))
y_hat2 = tf.nn.xw_plus_b(h_drop, FC_W, FC_b)
print(y_hat2.shape)
FC_W2 = tf.Variable(tf.truncated_normal([MAX_LABEL, MAX_LABEL], stddev=0.1))
FC_b2 = tf.Variable(tf.constant(0., shape=[MAX_LABEL]))
y_hat = tf.nn.xw_plus_b(y_hat2, FC_W2, FC_b2)

loss =  tf.nn.l2_loss(y_hat-batch_y) +0.001*tf.nn.l2_loss(FC_W)+0.001*tf.nn.l2_loss(FC_W2) + 0.0001*tf.nn.l2_loss(W)
# loss =   tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_hat, labels=batch_y)) +0.001*tf.nn.l2_loss(FC_W)+0.001*tf.nn.l2_loss(FC_W2) + 0.0001*tf.nn.l2_loss(W)

optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
prediction = tf.sigmoid(y_hat)
one_hot_prediction = multi_label_hot(prediction)

accuracy  =  get_metrics(batch_y,one_hot_prediction,77)

(?, 20, 308)
(?, 20, 128)
(?, 20)
<unknown>


In [51]:
saver = tf.train.Saver()
!mkdir checkpointDir

# Model Parameters
slim = tf.contrib.slim
sess=tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)
    
model_summary()

# To store training and test results for visualization

tr = []
ts = []

---------
Variables: name (type shape) [size]
---------
BLSTM_1/basic_lstm_cell/kernel:0 (float32_ref 436x512) [223232, bytes: 892928]
BLSTM_1/basic_lstm_cell/bias:0 (float32_ref 512) [512, bytes: 2048]
BLSTM_2/basic_lstm_cell/kernel:0 (float32_ref 436x512) [223232, bytes: 892928]
BLSTM_2/basic_lstm_cell/bias:0 (float32_ref 512) [512, bytes: 2048]
Variable:0 (float32_ref 2) [2, bytes: 8]
BLSTM_3/basic_lstm_cell/kernel:0 (float32_ref 256x512) [131072, bytes: 524288]
BLSTM_3/basic_lstm_cell/bias:0 (float32_ref 512) [512, bytes: 2048]
Variable_1:0 (float32_ref 128) [128, bytes: 512]
Variable_2:0 (float32_ref 128x308) [39424, bytes: 157696]
Variable_3:0 (float32_ref 308) [308, bytes: 1232]
Variable_4:0 (float32_ref 308x308) [94864, bytes: 379456]
Variable_5:0 (float32_ref 308) [308, bytes: 1232]
Total size of variables: 714106
Total bytes of variables: 2856424


A subdirectory or file checkpointDir already exists.


In [52]:
#sess.run(tf.global_variables_initializer())
print("Initialized! ")
target_names = ['a','b','c','d']
print("Start trainning")
start = time.time()

testA = 0
predsAr = []

# Training for 50 epochs
for e in range(epochs):   
# for e in range(5):

    epoch_start = time.time()
    print("Epoch %d start !" % (e + 1))
    # for x_batch, y_batch in zip(x_train, y_train, BATCH_SIZE):
    err = []
    preds = []
    trues = []

    # batch sizes for crimes and anomaly and prediction
    x_batch1 =[]
    x_batch2 = []
    y_batch1 = []
    
    # Recording error, prediction, truth values(for F1-scores)
    for i in range(len(x_train)-80):                                      # 292 - 80 = 212 
        i+=80
        x_batch = x_train[i:min(len(x_train)-1,timeSize+(i))]             # i=0 -> 0-min(292-1, 10+1)=11  -> 0-11 = 12  # i=1 -> 1-min(292-1, 10+1)=11  -> 1-11 = 10
        x_anomaly = x_train2[i:min(len(x_train)-1,timeSize+(i))]          # i=0 -> 0-min(292-1, 10+1)=11  -> 0-11 = 12  # i=1 -> 1-min(292-1, 10+1)=11  -> 1-11 = 10
        if len(x_batch) < timeSize:
          continue
        x_batch = x_batch
        x_anomaly = x_anomaly
        y_batch = x_train[min(len(x_train)-1,timeSize+(i))].T
        x_batch1.append(x_batch)
        x_batch2.append(x_anomaly)
        y_batch1.append(y_batch)
        if (i+1)% BATCH_SIZE >0:
          continue
        # print(np.array(x_batch1).shape)
        fd = {batch_x: x_batch1, anomaly_x: x_batch2, batch_y: y_batch1, keep_prob: KEEP_PROB}
        # print(y_batch)
        
        l, _, oht = sess.run([loss, optimizer, one_hot_prediction], feed_dict=fd)
        
        for j in range(BATCH_SIZE):
          # print(oht.shape)
          preds.extend(np.array(oht[j]).reshape(-1,4))
          trues.extend(np.array(y_batch1[j]).reshape(-1,4))
        x_batch1 =[]
        y_batch1 = []
        x_batch2 = []

        # sess.run(optimizer,feed_dict=fd)
        err.append(l)
    # print(sess.run(loss))
    epoch_finish = time.time()
    # print(preds)
    preds = np.array(preds)
    trues = np.array(trues)

    # f1 = f1_score(y_true=y_batch, y_pred=oht, average='weighted')
    f1 = f1_score(y_true=np.where(trues>0,1,0), y_pred=np.where(preds>0,1,0), average='micro')
    f2 = f1_score(y_true=np.where(trues>0,1,0), y_pred=np.where(preds>0,1,0), average='macro')
    tr.append([f1,f2])
    # print(f1)
    print("TRain :: ",np.mean(err)," : micro ",f1," : macro",f2," : ",epoch_finish-epoch_start)
    # print(classification_report(y_true=trues,y_pred=preds,target_names=target_names))

    ########### Predictions on test data and storing info for visualization ###########
    if True:
      preds = []
      trues = []
      x_batch1 =[]
      y_batch1 = []
      x_batch2 = []
      err = []
      for i in range(len(x_test)):
          # i+=100
          x_batch = x_test[i:min(len(x_test)-1,timeSize+(i))]
          x_anomaly = x_test2[i:min(len(x_test)-1,timeSize+(i))]
          if len(x_batch) < timeSize:
            continue
          x_batch = x_batch
          x_anomaly = x_anomaly
          y_batch = x_test[min(len(x_test)-1,timeSize+(i))].T
          x_batch1.append(x_batch)
          x_batch2.append(x_anomaly)
          y_batch1.append(y_batch)
          if (i+1)% BATCH_SIZE >0:
            continue
          fd = {batch_x: x_batch1,anomaly_x:x_batch2, batch_y: y_batch1, keep_prob: KEEP_PROB}
          
          l, acc,oht,weightSupport = sess.run([loss, accuracy,one_hot_prediction,weight_soft], feed_dict=fd)
          
          err.append(l)
          # sess.run(optimizer,feed_dict=fd)
          for j in range(BATCH_SIZE):
            # print(oht.shape)
            preds.extend(np.array(oht[j]).reshape(-1,4))
            trues.extend(np.array(y_batch1[j]).reshape(-1,4))

          x_batch1 =[]
          y_batch1 = []
          x_batch2 = []

      preds = np.array(preds)
      trues = np.array(trues)

    #   ##################################### BY ALIF #####################################
    #   # # save preds and trues in a pickle file
    #   # with open('../Chicago Dataset/preds_before_threshold_1d_only_crime_data.pkl', 'wb') as f:
    #   #   pickle.dump(preds, f)
    #   # with open('../Chicago Dataset/trues_before_threshold_1d_only_crime_data.pkl', 'wb') as f:
    #   #   pickle.dump(trues, f)
      
    #   # save preds and trues in a pickle file
    #   with open('../../Comparison/TEMPORAL_PRECISION/DEEPCRIME/Outputs/1d_wo_external/preds_before_threshold.pkl', 'wb') as f:
    #     pickle.dump(preds, f)
    #   with open('../../Comparison/TEMPORAL_PRECISION/DEEPCRIME/Outputs/1d_wo_external/trues_before_threshold.pkl', 'wb') as f:
    #     pickle.dump(trues, f)
    #   ###################################################################################

      y_pred = preds
      y_true = trues

    #   ##################################### BY ALIF #####################################
    #   # # save preds and trues in a pickle file
    #   # with open('../Chicago Dataset/preds_after_threshold_1d_only_crime_data.pkl', 'wb') as f:
    #   #   pickle.dump(y_pred, f)
    #   # with open('../Chicago Dataset/trues_after_threshold_1d_only_crime_data.pkl', 'wb') as f:
    #   #   pickle.dump(y_true, f)

      
      # save preds and trues in a pickle file
      with open('../HOLIDAY_ANALYSIS/pred_reg_easter.pkl', 'wb') as f:
        pickle.dump(y_pred, f)
      with open('../HOLIDAY_ANALYSIS/true_reg_easter.pkl', 'wb') as f:
        pickle.dump(y_true, f)
    #   ###################################################################################

      # with open('preds_NYC.pkl', 'wb') as f:
      #   pickle.dump(preds, f)
      # with open('trues_NYC.pkl', 'wb') as f:
      #   pickle.dump(trues, f)

      # f1 = f1_score(y_true=y_batch, y_pred=oht, average='weighted')
      f1 = f1_score(y_true=np.where(trues>0,1,0), y_pred=np.where(preds>0,1,0), average='micro')
      f2 = f1_score(y_true=np.where(trues>0,1,0), y_pred=np.where(preds>0,1,0), average='macro')

      if testA < f1:
        testA=f1
        save_path = saver.save(sess, "./modelM/model"+str(f1)[:5]+".ckpt")
        # print(classification_report(y_true=trues,y_pred=preds,target_names=target_names))
        predsAr.append(preds)
      ts.append([f1,f2])
      # print(f1)
      # print(np.mean(err)," : micro ",f1," : macro",f2," : ")
      # print(weightSupport)
      # print(classification_report(y_true=trues,y_pred=preds,target_names=target_names))

Initialized! 
Start trainning
Epoch 1 start !
TRain ::  7469.7144  : micro  0.6840546024367242  : macro 0.6569480710741321  :  1.8908653259277344
Epoch 2 start !
TRain ::  3527.7075  : micro  0.7225874450677052  : macro 0.693882503137584  :  1.7146399021148682
Epoch 3 start !
TRain ::  3484.1575  : micro  0.723207034146686  : macro 0.6948715637400052  :  1.7445487976074219
Epoch 4 start !
TRain ::  3383.9949  : micro  0.7227961764989862  : macro 0.6952419022815036  :  1.7096703052520752
Epoch 5 start !
TRain ::  3347.1863  : micro  0.7217967547908799  : macro 0.6942702334281582  :  1.6461844444274902
Epoch 6 start !
TRain ::  3340.1423  : micro  0.7210695932410068  : macro 0.6937265172876937  :  1.586057424545288
Epoch 7 start !
TRain ::  3314.952  : micro  0.7206492491971183  : macro 0.693516564118016  :  1.5635044574737549
Epoch 8 start !
TRain ::  3310.2417  : micro  0.7201757876706403  : macro 0.6932219072885124  :  1.713627815246582
Epoch 9 start !
TRain ::  3282.0427  : micro  0.